In [1]:
from Bio.Blast.Applications import NcbiblastnCommandline

def blast_sequence(sequence):
    blastn = NcbiblastnCommandline(query="input.fasta", db="aiptasia_db", out="results.xml", outfmt=5)
    stdout, stderr = blastn()
    # Parse the XML results and return (this part is not shown but can be done using Biopython's NCBIXML module)

user_sequence = "GTGAAAGGTATTTCACGACATGACGAGGACCAAGAACAGTGAAACTATCATTTGACACTATGTGGAGGAAGCCCATCTAAAAGCGATCTGTGATCGTTCCTTTTCAAGGATTGTATAGTGAAGGCCAGGTTACTTAGGCTTGGAATTATGTAACAACAACAAATGGCGCAGAAATTGTTCATGTACACCTCAGGCTTCTAGCACGGCATTACAGCTCAAGATTATGGCCAAAAACGTATAATATTGTCACATTTGGTAATAAAGGAGTTGTAACTAGAGGAATAATGCTAAGTGCTGGTTTACTAGCCAAAACCACAGATGAAGAATGGGGAAAAAAACTTACACCTCAACAATACCACGTTTGTAGGCAAAAGGGAACCGAGCCGCCTGGTCTGGTG"
results = blast_sequence(user_sequence)

ApplicationError: Non-zero return code 127 from 'blastn -out results.xml -outfmt 5 -query input.fasta -db aiptasia_db', message '/bin/sh: blastn: command not found'

In [8]:
from Bio import pairwise2
from Bio import SeqIO
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

def score_alignment(target_seq, record):
    align_score = pairwise2.align.globalxx(target_seq, record.seq, score_only=True)
    return record.id, align_score

def parallel_search_sequence(target_seq, database_filename, n_jobs=-1):
    # Read the entire database into a list (can be memory intensive for large databases)
    records = list(SeqIO.parse(database_filename, "fasta"))

    # Use joblib to parallelize the alignment scoring
    results = Parallel(n_jobs=n_jobs)(delayed(score_alignment)(target_seq, record) for record in tqdm(records))

    # Filter out None results
    return [result for result in results if result is not None]

database_filename = "data/aiptasia_genome.fna"
target_seq = "GTGAAAGGTATTTCACGACATGACGAGGACCAAGAACAGTGAAACTATCATTTGACACTATGTGGAGGAAGCCCATCTAAAAGCGATCTGTGATCGTTCCTTTTCAAGGATTGTATAGTGAAGGCCAGGTTACTTAGGCTTGGAATTATGTAACAACAACAAATGGCGCAGAAATTGTTCATGTACACCTCAGGCTTCTAGCACGGCATTACAGCTCAAGATTATGGCCAAAAACGTATAATATTGTCACATTTGGTAATAAAGGAGTTGTAACTAGAGGAATAATGCTAAGTGCTGGTTTACTAGCCAAAACCACAGATGAAGAATGGGGAAAAAAACTTACACCTCAACAATACCACGTTTGTAGGCAAAAGGGAACCGAGCCGCCTGGTCTGGTG"
aligned_records = parallel_search_sequence(target_seq, database_filename)

  0%|          | 0/4312 [00:00<?, ?it/s]

/Users/daxledesma/miniconda3/envs/siena/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
/Users/daxledesma/miniconda3/envs/siena/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
/Users/daxledesma/miniconda3/envs/siena/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release 

KeyboardInterrupt: 

In [ ]:
from Bio import pairwise2
from Bio import SeqIO
from joblib import Parallel, delayed

def gc_content(seq):
    return (seq.count("G") + seq.count("C")) / len(seq) * 100

def score_alignment(target_seq, motif):
    if 30 <= gc_content(target_seq) <= 55:  # Check the GC content criteria
        align_score = pairwise2.align.globalxx(target_seq, motif, score_only=True)
        if align_score == len(motif):  # Since the length of the motif is 21, a perfect score would be 21.
            return motif
    return None

def parallel_search_motifs(target_seq, database_filename, n_jobs=-1):
    # Extract all 21 nt motifs from the database
    records = list(SeqIO.parse(database_filename, "fasta"))
    motifs = []
    for record in tqdm(records):
        for i in range(len(record.seq) - 21 + 1):
            motifs.append(record.seq[i:i+21])

    # Use joblib to parallelize the alignment scoring
    matching_motifs = Parallel(n_jobs=n_jobs)(delayed(score_alignment)(target_seq, motif) for motif in tqdm(motifs))

    # Filter out None results
    return [motif for motif in matching_motifs if motif is not None]

database_filename = "data/aiptasia_genome.fna"
target_seq = "GTGAAAGGTATTTCACGACATGACGAGGACCAAGAACAGTGAAACTATCATTTGACACTATGTGGAGGAAGCCCATCTAAAAGCGATCTGTGATCGTTCCTTTTCAAGGATTGTATAGTGAAGGCCAGGTTACTTAGGCTTGGAATTATGTAACAACAACAAATGGCGCAGAAATTGTTCATGTACACCTCAGGCTTCTAGCACGGCATTACAGCTCAAGATTATGGCCAAAAACGTATAATATTGTCACATTTGGTAATAAAGGAGTTGTAACTAGAGGAATAATGCTAAGTGCTGGTTTACTAGCCAAAACCACAGATGAAGAATGGGGAAAAAAACTTACACCTCAACAATACCACGTTTGTAGGCAAAAGGGAACCGAGCCGCCTGGTCTGGTG"
aligned_motifs = parallel_search_motifs(target_seq, database_filename)

  0%|          | 0/4312 [00:00<?, ?it/s]

In [ ]:
import re

class siRNAApp:
    def __init__(self):
        self.sirna_name = None
        self.target_gene_sequence = ""
        self.selection_criteria = "default_criteria"

    def remove_spaces_and_numbers(self, sequence):
        return re.sub(r'[\s\d]', '', sequence)

    def get_input_data(self):
        self.sirna_name = input("Enter your siRNA name for reference (optional): ")
        sequence_input = input("Paste the sequence of your target gene: ")
        self.target_gene_sequence = self.remove_spaces_and_numbers(sequence_input)
        criteria = input("Enter the desired selection criteria (or type 'default' to use default settings): ")

    def process_sequence(self):
        print(f"Processing sequence for siRNA {self.sirna_name}...")

    def check_specificity(self):
        # Mock-up for checking specificity
        print(f"Checking specificity for {self.sirna_name}...")
        # Link to a database and implement a function to check for non-unique sequences

    def check_off_target_effects(self):
        # Mock-up for checking off-target effects
        print(f"Checking off-target effects for {self.sirna_name}...")
        # Link to an miRNA SEED database and check for potential off-target effects

    def run(self):
        self.get_input_data()
        self.process_sequence()
        self.check_specificity()
        self.check_off_target_effects()
        print("siRNA design complete.")

if __name__ == "__main__":
    app = siRNAApp()
    app.run()


In [6]:
import re
from Bio import SeqUtils
from Bio.Blast import NCBIWWW, NCBIXML

def select_target_region(sequence, length=21, position_from_3UTR=50):
    '''Select a target region near the 3' UTR.'''
    return sequence[-(position_from_3UTR + length):-position_from_3UTR]

def check_gc_content(sequence):
    '''Check GC content. It should be between 30% and 52%.'''
    gc_content = SeqUtils.gc(sequence)
    return 30 <= gc_content <= 52

def avoid_certain_patterns(sequence):
    '''Avoid sequences with more than four consecutive same nucleotides.'''
    patterns = ["AAAAA", "TTTTT", "GGGGG", "CCCCC"]
    for pattern in patterns:
        if pattern in sequence:
            return False
    return True

def check_thermodynamics(sequence):
    '''Check thermodynamic stability at 5' end vs 3' end. Basic example.'''
    five_prime = sequence[:5]
    three_prime = sequence[-5:]
    return SeqUtils.gc(five_prime) < SeqUtils.gc(three_prime)

def avoid_off_target(sequence):
    '''Check against BLAST database for off-target effects. This is a mockup.'''
    # Perform BLAST search (this may result in an error if there are issues connecting to NCBI or if there are too many requests)
    result_handle = NCBIWWW.qblast("blastn", "nt", sequence)
    blast_record = NCBIXML.read(result_handle)
    
    # If there are matches to other sequences, return False
    if blast_record.alignments:
        return False
    return True

def design_siRNA(sequence):
    '''Design an siRNA given the input sequence.'''
    target = select_target_region(sequence)
    
    if not check_gc_content(target):
        print("Failed GC content check.")
        return None
    
    if not avoid_certain_patterns(target):
        print("Failed pattern avoidance check.")
        return None
    
    if not check_thermodynamics(target):
        print("Failed thermodynamics check.")
        return None
    
    if not avoid_off_target(target):
        print("Potential off-target effects detected.")
        return None
    
    return target

# Example usage
sequence = "GTGAAAGGTATTTCACGACATGACGAGGACCAAGAACAGTGAAACTATCATTTGACACTATGTGGAGGAAGCCCATCTAAAAGCGATCTGTGATCGTTCCTTTTCAAGGATTGTATAGTGAAGGCCAGGTTACTTAGGCTTGGAATTATGTAACAACAACAAATGGCGCAGAAATTGTTCATGTACACCTCAGGCTTCTAGCACGGCATTACAGCTCAAGATTATGGCCAAAAACGTATAATATTGTCACATTTGGTAATAAAGGAGTTGTAACTAGAGGAATAATGCTAAGTGCTGGTTTACTAGCCAAAACCACAGATGAAGAATGGGGAAAAAAACTTACACCTCAACAATACCACGTTTGTAGGCAAAAGGGAACCGAGCCGCCTGGTCTGGTG"
result = design_siRNA(sequence)
if result:
    print(f"Designed siRNA: {result}")
else:
    print("Failed to design siRNA based on criteria.")

Failed GC content check.
Failed to design siRNA based on criteria.
